# **Import Libraries**

In [1]:
import joblib
import numpy as np
import pandas as pd
import tensorflow as tf
from Model import HybridRecommendation
from TF_Object import L2NormalizeLayer
from TF_Object import CollaborativeFilteringLayer
from TF_Object import CollaborativeFilteringModel

# **Prepare Dataset**

In [2]:
# Load all dataset in Pandas DataFrame
tourism_df = pd.read_csv("../Dataset/new_tourism_with_id_links_3.csv")
distance_df = pd.read_csv("../Dataset/tourist_spots_distance.csv")

In [3]:
tourism_df.head(3)

,Place_Id,Place_Name,Category,Description,City,Price,Rating,Time_Minutes,Coordinate,Lat,Long,Img_Path_0,Img_Path_1,Img_Path_2
0,1,Monumen Nasional,"Budaya, Situs Sejarah",Monumen Nasional atau yang populer disingkat d...,Jakarta,20000,"4,6",15.0,"{'lat': -6.1753924, 'lng': 106.8271528}",-6.175392,106.827153,images_output\Monumen Nasional\Monumen Nasiona...,images_output\Monumen Nasional\Monumen Nasiona...,images_output\Monumen Nasional\Monumen Nasiona...
1,2,Kota Tua,Budaya,"Kota tua di Jakarta, yang juga bernama Kota Tu...",Jakarta,0,"4,6",90.0,"{'lat': -6.137644799999999, 'lng': 106.8171245}",-6.137645,106.817125,images_output\Kota Tua\Kota Tua_0.jpg,images_output\Kota Tua\Kota Tua_1.jpg,NaN
2,3,Dunia Fantasi,"Taman Hiburan, Bahari",Dunia Fantasi atau disebut juga Dufan adalah t...,Jakarta,270000,"4,6",360.0,"{'lat': -6.125312399999999, 'lng': 106.8335377}",-6.125312,106.833538,images_output\Dunia Fantasi\Dunia Fantasi_0.jpg,images_output\Dunia Fantasi\Dunia Fantasi_1.png,images_output\Dunia Fantasi\Dunia Fantasi_2.jpg


In [4]:
distance_df.sample(3)

,Place_Id_Source,Place_Id_Target,Distance
143045,329,38,113.34
75784,174,358,87.11
48060,111,101,14.77


# **Initiate Important Instance**

In [6]:
# Load Vector Data
user_data_vecs = np.load('../Vector/user_vector.npy')        # Example of transformation result from user DB
tourism_data_vecs = np.load('../Vector/tourism_vector.npy')  # Example of transformation result from tourism DB

# Load Y_mean for denormalize prediction results
Y_mean = np.load('../Vector/Y_mean.npy')

# Load Scalers
user_scaler = joblib.load('../Scaler/user_scaler.gz')
tourism_scaler = joblib.load('../Scaler/tourism_scaler.gz')
target_scaler = joblib.load('../Scaler/target_scaler.gz')

# Load Collaborative Filtering Model
cofi_model = tf.keras.models.load_model('Model/collaborative_filtering.h5', custom_objects={
    'CollaborativeFilteringModel': CollaborativeFilteringModel,
    'CollaborativeFilteringLayer': CollaborativeFilteringLayer
})

# Load Content-Based Model
cb_model = tf.keras.models.load_model('Model/content_based.h5', custom_objects={
    'L2NormalizeLayer': L2NormalizeLayer
})

# Instantiate Hybrid Model
hybrid_model = HybridRecommendation(
    cofi_model=cofi_model,
    cb_model=cb_model,
    user_scaler=user_scaler,
    tourism_scaler=tourism_scaler,
    target_scaler=target_scaler,
    Y_mean=Y_mean
)

# **Hybrid Recommendation**

## **Normal Hybrid Reccomendation**

In [ ]:
# Get 10 hybrid recommendations for user_id = 120

# Prepare user data
user_id = 120
num_recom = 10
current_user_data = user_data_vecs[user_id]

# Get (normal) hybrid recommendation for that user
recommended_id = hybrid_model.get_recommendation(
    user_id,
    current_user_data,
    tourism_data_vecs
)

# Displays recommended results from highest to lowest
result_1 = tourism_df.loc[tourism_df['Place_Id'].isin(recommended_id)].\
    set_index('Place_Id').reindex(recommended_id).reset_index()
result_1.iloc[:,:6].head(num_recom)

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


,Place_Id,Place_Name,Category,Description,City,Price
0,97,Monumen Yogya Kembali,"Budaya, Situs Sejarah",Museum Monumen Yogya Kembali (bahasa Jawa: ꦩꦺꦴ...,Yogyakarta,15000
1,115,Monumen Sanapati,"Budaya, Situs Sejarah",Monumen Sanapati dibangun untuk memeringati 50...,Yogyakarta,15000
2,91,Situs Warungboto,"Budaya, Situs Sejarah",Situs Warungboto atau Pesanggrahan Rejawinangu...,Yogyakarta,0
3,1,Monumen Nasional,"Budaya, Situs Sejarah",Monumen Nasional atau yang populer disingkat d...,Jakarta,20000
4,14,Pulau Pelangi,Bahari,Pulau Pelangi adalah sebuah pulau yang terleta...,Jakarta,900000
5,44,Monumen Selamat Datang,"Budaya, Situs Sejarah",Monumen Selamat Datang adalah sebuah monumen y...,Jakarta,0
6,41,Museum Bahari Jakarta,"Budaya, Situs Sejarah",Museum Bahari adalah museum yang menyimpan kol...,Jakarta,2000
7,334,Chingu Cafe Little Seoul,"Taman Hiburan, Bahari",Selain populer karena memiliki pemandangan yan...,Bandung,50000
8,171,Candi Ijo,Budaya,"Candi Ijo (bahasa Jawa: ꦕꦤ꧀ꦝꦶ​ꦆꦗꦺꦴ, translit. ...",Yogyakarta,5000
9,125,Alun-alun Utara Keraton Yogyakarta,"Budaya, Situs Sejarah",Alun-alun utara atau dalam Bahasa Jawa disebut...,Yogyakarta,0


## **Distance-Based Hybrid Recommendation**

In [ ]:
# Get 10 hybrid recommendations for user_id = 120,
# near Sanghyang Heuleut tourist spot (Bandung)

# Prepare data
user_id = 120
num_recom = 10
choosen_spot_id = 300  # Sanghyang Heuleut (Bandung)
current_user_data = user_data_vecs[user_id]
choosen_spot_distance_df = distance_df.loc[distance_df['Place_Id_Source'] == choosen_spot_id]

# Get (distance-based) hybrid recommendation for that user
recommended_id = hybrid_model.get_recommendation(
    user_id,
    current_user_data,
    tourism_data_vecs,
    choosen_spot_id,
    choosen_spot_distance_df
)

# Displays recommended results from highest to lowest (free to take as much tourism data as you want)
result_2 = tourism_df.loc[tourism_df['Place_Id'].isin(list(recommended_id.keys()))].\
    set_index('Place_Id').reindex(recommended_id).reset_index()
result_2['Distance'] = list(recommended_id.values())
result_2.loc[:, ['Place_Id', 'Place_Name', 'Category', 'Description', 'City', 'Price', 'Distance']].head(num_recom)

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


,Place_Id,Place_Name,Category,Description,City,Price,Distance
0,334,Chingu Cafe Little Seoul,"Taman Hiburan, Bahari",Selain populer karena memiliki pemandangan yan...,Bandung,50000,29.69
1,285,Taman Sejarah Bandung,"Budaya, Situs Sejarah",Taman Sejarah Bandung adalah taman yang terlet...,Bandung,0,29.78
2,277,Peta Park,"Taman Hiburan, Bahari",Bandung seperti tidak henti menawarkan tempat ...,Bandung,15000,27.85
3,290,Museum Pendidikan Nasional,"Budaya, Taman Hiburan",Museum Pendidikan Nasional Universitas Pendidi...,Bandung,5000,27.88
4,229,Rabbit Town,"Taman Hiburan, Bahari",Rabbit Town Bandung merupakan salah satu wisat...,Bandung,45000,29.59
5,260,Museum Pos Indonesia,"Budaya, Situs Sejarah",Museum Pos Indonesia telah ada sejak masa Hind...,Bandung,0,30.77
6,321,Glamping Lakeside Rancabali,"Taman Hiburan, Bahari",Glamping Lakeside Rancabali menawarkan tempat ...,Bandung,30000,32.39
7,287,Puspa Iptek Sundial,Taman Hiburan,Puspa Iptek Sundial adalah wahana pendidikan y...,Bandung,25000,16.96
8,254,Teras Cikapundung BBWS,Taman Hiburan,Teras Cikapundung Bandung sebelumnya merupakan...,Bandung,0,29.23
9,216,Farm House Susu Lembang,Taman Hiburan,"Sebagai salah satu tempat wisata lembang baru,...",Bandung,30000,29.48
